<a href="https://colab.research.google.com/github/sugarforever/LangChain-Advanced/blob/main/langchain_with_openai_assistants_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Assistants API with LangChain

OpenAI finally released the **Assistants** API. It allows users to build AI assistants within the applications.

An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries. The **Assistants** API currently supports three types of tools:
- Code Interpreter
- Retrieval
- Function calling

In this tutorial, I will share with you 2 ways of using **Assistants** API:

1. Command line tool `curl`
2. `LangChain`

You can surely use the official OpenAI Python SDK. It's just not covered in this tutorial.

In principle, to interact with an Assistant, the following steps should be followed:

1. Create an `Assistant`
2. Create a `Thread`
3. Submit a `Run`
4. Query the `Run` status
5. Retrieve the `Assistant`'s response (message)

## Play with `curl`

Before you run the following commands, make sure you have a valid **OPENAI_API_KEY** environmental variable set.

### 1. Create an Assistant

```shell
curl "https://api.openai.com/v1/assistants" \
  -u:$OPENAI_API_KEY \
  -H "Content-Type: application/json" \
  -H "OpenAI-Beta: assistants=v1" \
  -d '{
    "instructions": "You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
    "name": "Math Tutor",
    "tools": [{"type": "code_interpreter"}],
    "model": "gpt-4"
  }'
```

You will be given an assistant id in this API response.

```json
{
  "id": "asst_M1qNqUsUIeItZ8f7JG6FLHTy",
  "object": "assistant",
  "created_at": 1699486594,
  "name": "Math Tutor",
  "description": null,
  "model": "gpt-4",
  "instructions": "You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "file_ids": [],
  "metadata": {}
}%         
```

### 2. Create a Thread

A `Thread` represents a conversation between an Assistant and a user.

```shell
curl https://api.openai.com/v1/threads \
  -u:$OPENAI_API_KEY \
  -H 'Content-Type: application/json' \
  -H 'OpenAI-Beta: assistants=v1' \
  -d '{
    "messages": [
      {
          "role": "user",
          "content": "Work out the sum of 1, 2 and 3"
      }
    ]
  }'
```

You will be given a Thread id in this API response.

```json
{
  "id": "thread_lEH83Q4bKV3hv4Y9yECcjfgy",
  "object": "thread",
  "created_at": 1699486702,
  "metadata": {}
}
```

### 3. Submit a Run

Submit a Run with the `thread id` and `assistant id` to `run` the conversation.

```shell
curl https://api.openai.com/v1/threads/<thread_id>/runs \
  -u:$OPENAI_API_KEY \
  -H 'Content-Type: application/json' \
  -H 'OpenAI-Beta: assistants=v1' \
  -d '{
    "assistant_id": "asst_M1qNqUsUIeItZ8f7JG6FLHTy"
  }'
```

You will be given a Run id in this API response.

```json
{
  "id": "run_in35OoYZNo0osZnQOQrAHaNo",
  "object": "thread.run",
  "created_at": 1699486777,
  "assistant_id": "asst_M1qNqUsUIeItZ8f7JG6FLHTy",
  "thread_id": "thread_lEH83Q4bKV3hv4Y9yECcjfgy",
  "status": "queued",
  "started_at": null,
  "expires_at": 1699487377,
  "cancelled_at": null,
  "failed_at": null,
  "completed_at": null,
  "last_error": null,
  "model": "gpt-4",
  "instructions": "You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "file_ids": [],
  "metadata": {}
}
```

### 4. Query the Run status

```shell
curl https://api.openai.com/v1/threads/<thread_id>/runs/<run_id> \
  -u:$OPENAI_API_KEY \
  -H 'OpenAI-Beta: assistants=v1'
```

You will see the status of the Run in the API response.

```json
{
  "id": "run_in35OoYZNo0osZnQOQrAHaNo",
  "object": "thread.run",
  "created_at": 1699486777,
  "assistant_id": "asst_M1qNqUsUIeItZ8f7JG6FLHTy",
  "thread_id": "thread_lEH83Q4bKV3hv4Y9yECcjfgy",
  "status": "completed",
  "started_at": 1699486777,
  "expires_at": null,
  "cancelled_at": null,
  "failed_at": null,
  "completed_at": 1699486783,
  "last_error": null,
  "model": "gpt-4",
  "instructions": "You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "file_ids": [],
  "metadata": {}
}%
```

### 5. Retrieve the Assistant's response

The Assistant's response will be the latest message in the following API response:

```shell
curl https://api.openai.com/v1/threads/<thread_id>/messages \
  -u:$OPENAI_API_KEY \
  -H "Content-Type: application/json" \
  -H "OpenAI-Beta: assistants=v1"
```

Sample response:

```json
{
  "object": "list",
  "data": [
    {
      "id": "msg_bCgflETMYexEUEGqHkwNgN6Z",
      "object": "thread.message",
      "created_at": 1699486782,
      "thread_id": "thread_lEH83Q4bKV3hv4Y9yECcjfgy",
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": {
            "value": "The sum of 1, 2, and 3 is 6.",
            "annotations": []
          }
        }
      ],
      "file_ids": [],
      "assistant_id": "asst_M1qNqUsUIeItZ8f7JG6FLHTy",
      "run_id": "run_in35OoYZNo0osZnQOQrAHaNo",
      "metadata": {}
    },
    {
      "id": "msg_I4xv8bsfy5evvEC8K4MVZar3",
      "object": "thread.message",
      "created_at": 1699486702,
      "thread_id": "thread_lEH83Q4bKV3hv4Y9yECcjfgy",
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": {
            "value": "Work out the sum of 1, 2 and 3",
            "annotations": []
          }
        }
      ],
      "file_ids": [],
      "assistant_id": null,
      "run_id": null,
      "metadata": {}
    }
  ],
  "first_id": "msg_bCgflETMYexEUEGqHkwNgN6Z",
  "last_id": "msg_I4xv8bsfy5evvEC8K4MVZar3",
  "has_more": false
}%
```

## Play with LangChain

LangChain dev team has been responding to OpenAI changes proactively. The latest **RC** version of LangChain has already supported **Assistants** API.

Make sure you use at least version `0.0.331rc2` of LangChain to work with Assistants API.

Currently LangChain is delivering the Assistants support by `langchain-experimental` package.

The only class you need is just **OpenAIAssistantRunnable**, which makes code much cleaner.

Let's see below.

In [17]:
import os
os.environ["OPENAI_API_KEY"] = 'your valid OpenAI API key with GPT-4 enabled'

In [ ]:
!pip install -U -q "langchain==0.0.331rc2" langchain-experimental "openai>=1.1"

In [14]:
from langchain_experimental.openai_assistant import OpenAIAssistantRunnable

In [18]:
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4"
)

In [19]:
output = interpreter_assistant.invoke({"content": "Work out the sum of 1, 2 and 3"})
output

[ThreadMessage(id='msg_RnANxbiBnHsm08qNLOZiMLgX', assistant_id='asst_ZYhTHQEq3oxx9EM8a7TyE4Ac', content=[MessageContentText(text=Text(annotations=[], value='The sum of 1, 2 and 3 is 6.'), type='text')], created_at=1699487072, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_x9f7CXuvuAKGbGwGktLf1ghY', thread_id='thread_Fe5tZZGTBhJShOaBPDAbqOjz')]